In [1]:
#load package
.libPaths(c("/cluster/home/liuhengxin/software/miniconda3_2/envs/r-kernel/lib/R/library",
           "/cluster/home/liuhengxin/software/miniconda3/envs/r4.1.2/lib/R/library",
           "/cluster/home/liuhengxin/software/miniconda3_2/envs/Rsplot/lib/R/library"))
library(data.table)
library(qs)
library(GenomicRanges)
library(GenomicFeatures)
library(rtracklayer)
library(dplyr)
library(ggbio)
library(reshape2)
library(ggsci)
library(viridis)
library(ggpubr)
library(ggh4x)
library(ggtranscript)
library(khroma)
library(bambu)
setwd("/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/")
MySplit = function(str,sep, n){
  return(unlist(lapply(strsplit(str,sep),"[[",n)))
}

qs 0.25.5

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: I

# load data

In [2]:
fsraw.t42 = qread("data/reads_full_anotation/T42_main_data_read_total_annotation_delete_multiread_total_hubselect_25_12_10.qs")
fsraw.t47 = qread("data/reads_full_anotation/T47_main_data_read_total_annotation_delete_multiread_total_hubselect_25_12_10.qs")
fsraw.t44 = qread("data/reads_full_anotation/main_data_read_total_annotation_fil_delete_multiread_withsample_25_10_20.qs")
rt_vm116 = qread("/cluster/home/liuhengxin/P1_brain_nanopore/processed_data/VM_analysis/Revo/VM11LPT6_Y00983LA_trans_total_anno_withcid_filter_25_12_02.qs")
rt_me01 = qread("/cluster/home/liuhengxin/P1_brain_nanopore/processed_data/VM_analysis/Revo/ME012_Y00986G1_trans_total_anno_withcid_filter_25_12_02.qs")
isosigls.t44 = qread("data/dtu_analysis/dtu_isoform_ls_fsraw_cor_25_10_10.qs")
isosigls.t42 = qread("data/dtu_analysis/T42_dtu_isoform_ls_fsraw_cor_25_12_16.qs")
isosigls.t47 = qread("data/dtu_analysis/T47_dtu_isoform_ls_fsraw_cor_25_12_16.qs")

# Table1 isoform count and gtf count

In [11]:
datas1 = list()

In [6]:
gtfinfo.t44 = qread("ref/bambu_extended_annotations_transinfo.qs")
gtfinfo.t44 = as.data.frame(gtfinfo.t44$transinfo)
head(gtfinfo.t44)

,TXNAME,GENEID,NDR,novelGene,novelTranscript,txClassDescription,readCount,relReadCount,relSubsetCount,txid,eqClassById,fullLengthCounts
,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<chr>,<int>,<dbl>,<dbl>,<int>,<list>,<dbl>
BambuTx1,BambuTx1,PGBD2,0.5484456,FALSE,TRUE,newLastJunction:newLastExon,127,0.413680782,1.0000000,1,1,129.00000
BambuTx2,BambuTx2,ZNF692,0.9015748,FALSE,TRUE,newWithin,7,0.003280225,0.5384615,2,"2, 227827",5.00001
BambuTx3,BambuTx3,ZNF692,0.8325641,FALSE,TRUE,newJunction,19,0.008903468,0.7916667,3,"3, 192203",19.00000
BambuTx4,BambuTx4,LOC107130781,0.2581615,FALSE,TRUE,allNew,629,0.940209268,1.0000000,4,4,629.00000
BambuTx5,BambuTx5,SH3BP5L,0.9214234,FALSE,TRUE,newWithin,9,0.005717916,1.0000000,5,5,9.00000
BambuTx6,BambuTx6,NLRP3,0.7253364,FALSE,TRUE,allNew,33,0.160194175,1.0000000,6,6,38.00000


In [7]:
sumfull = read.csv("ref/gtf_filter_final_with_newname_25_10_09.csv")
head(sumfull)

,transcript_id,gene_id,trantype,transnew,fulllength,transnew2
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,BambuTx1,PGBD2,novel,PGBD2-WeiLab-1,599,PGBD2-FS-1
2,BambuTx10,ZNF669,novel,ZNF669-WeiLab-1,203,ZNF669-FS-1
3,BambuTx1000,USP21,novel,USP21-WeiLab-2,413,USP21-FS-2
4,BambuTx100007,LOC102142787,novel,LOC102142787-WeiLab-1,105,LOC102142787-FS-1
5,BambuTx100045,EXOC3L4,novel,EXOC3L4-WeiLab-1,72,EXOC3L4-FS-1
6,BambuTx100047,EXOC3L4,novel,EXOC3L4-WeiLab-2,37,EXOC3L4-FS-2


In [8]:
gtfinfo.res = merge(gtfinfo.t44[,c("TXNAME","GENEID","NDR","novelGene","novelTranscript","txClassDescription")],
                   sumfull[,c("transcript_id","transnew2","fulllength")],by.x = "TXNAME",by.y = "transcript_id")
nrow(gtfinfo.res)

[1] 38130

In [9]:
gtfinfo.res$transtype = "NNC"
gtfinfo.res[gtfinfo.res$txClassDescription %in% c("newWithin","newLastExon","newFirstExon","newFirstExon:newLastExon"),"transtype"] = "NIC"
gtfinfo.res[gtfinfo.res$txClassDescription == "annotation","transtype"] = "Known"
head(gtfinfo.res)

,TXNAME,GENEID,NDR,novelGene,novelTranscript,txClassDescription,transnew2,fulllength,transtype
,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<chr>,<chr>,<int>,<chr>
1,BambuTx1,PGBD2,0.5484456,FALSE,TRUE,newLastJunction:newLastExon,PGBD2-FS-1,599,NNC
2,BambuTx10,ZNF669,0.7928645,FALSE,TRUE,newLastJunction:newJunction:newLastExon,ZNF669-FS-1,203,NNC
3,BambuTx1000,USP21,0.7558931,FALSE,TRUE,newJunction,USP21-FS-2,413,NNC
4,BambuTx100007,LOC102142787,0.7591990,FALSE,TRUE,allNew,LOC102142787-FS-1,105,NNC
5,BambuTx100045,EXOC3L4,0.7991295,FALSE,TRUE,newFirstJunction:newFirstExon,EXOC3L4-FS-1,72,NNC
6,BambuTx100047,EXOC3L4,0.7364341,FALSE,TRUE,newLastJunction:newJunction,EXOC3L4-FS-2,37,NNC


In [12]:
colnames(gtfinfo.res)[7] = "transcript_id"
summary(gtfinfo.res$fulllength)
datas1$gtfinfo = gtfinfo.res

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      1       7      60    5683     433 3852656 

In [21]:
fsraw.vm = rt_vm116 %>% group_by(readid) %>% filter(length(unique(transcript_id)) == 1)
fsraw.me = rt_me01 %>% group_by(readid) %>% filter(length(unique(transcript_id)) == 1)

In [24]:
##T42 stat raw count for every sample
transt.t42 = fsraw.t42 %>% group_by(gene_id,transcript_id) %>% summarise(count = n())
transt.t42$platform = "Fullscope-Pacbio"
transt.t42$sample = "Macaque1-2"

#T44
transt.t44 = fsraw.t44 %>% group_by(group,gene_id,transcript_id) %>% summarise(count = n())
renamedf = data.frame("group" = c('ontf','cycf','pacbio','ontd'),
                      "platform" = c("Fullscope-ONT","Fullscope-Cyclone",
                                     "Fullscope-Pacbio","Direct-ONT"))
transt.t44 = merge(transt.t44,renamedf,by = "group")
transt.t44$sample = "Macaque1-1"
transt.t44 = transt.t44[,c("gene_id","transcript_id","platform","sample","count")]

#T47
transt.t47 = fsraw.t47 %>% group_by(group,gene_id,transcript_id) %>% summarise(count = n())
renamedf = data.frame("group" = c('Pacbio','ONT','Cyclone'),
                      "platform" = c("Fullscope-Pacbio","Fullscope-ONT","Fullscope-Cyclone"))
transt.t47 = merge(transt.t47,renamedf,by = "group")
transt.t47$sample = "Macaque1-3"
transt.t47 = transt.t47[,c("gene_id","transcript_id","platform","sample","count")]

#VM
transt.vm = fsraw.vm %>% group_by(gene_id,transcript_id) %>% summarise(count = n())
transt.vm$platform = "Fullscope-Pacbio"
transt.vm$sample = "Macaque2-1"

#ME
transt.me = fsraw.me %>% group_by(gene_id,transcript_id) %>% summarise(count = n())
transt.me$platform = "Fullscope-Pacbio"
transt.me$sample = "Macaque3-1"

transtl = rbind(transt.t42,transt.t44,transt.t47,transt.vm,transt.me)
head(transtl)

`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'group', 'gene_id'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'group', 'gene_id'. You can override using
the `.groups` argument.
`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.


gene_id,transcript_id,count,platform,sample
<chr>,<chr>,<int>,<chr>,<chr>
AAAS,AAAS-FS-1,12,Fullscope-Pacbio,Macaque1-2
AAAS,AAAS-FS-2,6,Fullscope-Pacbio,Macaque1-2
AAAS,XM_005571000.2,6,Fullscope-Pacbio,Macaque1-2
AAAS,XM_015430971.1,2,Fullscope-Pacbio,Macaque1-2
AAAS,XM_015430972.1,4,Fullscope-Pacbio,Macaque1-2
AACS,NM_001284010.1,70,Fullscope-Pacbio,Macaque1-2


In [26]:
transtmx = dcast(transtl,gene_id+transcript_id~sample+platform,value.var = "count",fill = 0,fun.aggregate = sum)
head(transtmx)
dim(transtmx)

,gene_id,transcript_id,Macaque1-1_Direct-ONT,Macaque1-1_Fullscope-Cyclone,Macaque1-1_Fullscope-ONT,Macaque1-1_Fullscope-Pacbio,Macaque1-2_Fullscope-Pacbio,Macaque1-3_Fullscope-Cyclone,Macaque1-3_Fullscope-ONT,Macaque1-3_Fullscope-Pacbio,Macaque2-1_Fullscope-Pacbio,Macaque3-1_Fullscope-Pacbio
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,A4GALT,XM_015457101.1,0,28,49,59,0,2,0,0,4,0
2,A4GALT,XM_015457102.1,2,2,0,0,0,0,0,0,0,0
3,AAAS,AAAS-FS-1,3,49,92,35,12,3,0,16,0,0
4,AAAS,AAAS-FS-2,9,3,18,7,6,3,0,12,3,0
5,AAAS,XM_005571000.2,3,375,589,11,6,0,0,18,3,19
6,AAAS,XM_015430971.1,0,3,4,3,2,0,0,2,0,0


[1] 36851    12

In [29]:
datas1$transtmx = transtmx

In [30]:
#library(xlsx)
# install.packages("writexl")
writexl::write_xlsx(datas1, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables1_26_01_07.xlsx")

# Table2 isoform DTU calculation

In [32]:
names(isosigls.t44)
names(isosigls.t47)
names(isosigls.t42)
i = 1
isosigi.t44 = isosigls.t44[[i]]
head(isosigi.t44)

[1] "layer"       "region"      "lobe"        "subclass"    "class"      
[6] "subcortical"

[1] "layer"       "region"      "lobe"        "subclass"    "class"      
[6] "subcortical"

[1] "layer"       "region"      "lobe"        "subclass"    "class"      
[6] "subcortical"

,gene_id,transcript_id,areatr,count,genecount,relative,meanrelative,otherrelative,dr,sig,pvalue,padj
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,AAAS,AAAS-FS-1,l1,4,24,0.16666667,0.1495626,0.1461418,0.02052482,N,0.0001939086,0.002006998
2,AAAS,AAAS-FS-1,l2,3,41,0.07317073,0.1495626,0.1648410,-0.09167030,N,0.0001939086,0.002006998
3,AAAS,AAAS-FS-1,l3,45,346,0.13005780,0.1495626,0.1534636,-0.02340581,N,0.0001939086,0.002006998
4,AAAS,AAAS-FS-1,l4,14,66,0.21212121,0.1495626,0.1370509,0.07507028,N,0.0001939086,0.002006998
5,AAAS,AAAS-FS-1,l5,10,153,0.06535948,0.1495626,0.1664033,-0.10104381,N,0.0001939086,0.002006998
6,AAAS,AAAS-FS-1,l6,28,112,0.25000000,0.1495626,0.1294752,0.12052482,Up,0.0001939086,0.002006998


In [39]:
data2ls = list()

for(i in 1:6){
    isosigi.t44 = isosigls.t44[[i]]
    isosigi.t47 = isosigls.t47[[i]]
    isosigi.t42 = isosigls.t42[[i]]
    
    hubtrans = isosigi.t44[isosigi.t44$sig != "N",]$transcript_id
    isosigi.t44 = isosigi.t44[isosigi.t44$transcript_id %in% hubtrans,]
    isosigi.t47 = isosigi.t47[isosigi.t47$transcript_id %in% hubtrans,]
    isosigi.t42 = isosigi.t42[isosigi.t42$transcript_id %in% hubtrans,]
    commonid1 = c("gene_id","transcript_id","areatr","sig","pvalue","padj",
                  "count","genecount","relative","otherrelative","dr")
    commonid2 = c("gene_id","transcript_id","areatr","count","genecount","relative","otherrelative","dr")
    
    isosigi = merge(isosigi.t44[,commonid1],isosigi.t42[,commonid2],by = c("gene_id","transcript_id","areatr"),
                    suffixes = c(".M1-1",".M1-2"))
    isosigi.t47 = isosigi.t47[,commonid2]
    colnames(isosigi.t47)[4:8] = paste0(colnames(isosigi.t47)[4:8],".M1-3")
    isosigi = merge(isosigi,isosigi.t47,by = c("gene_id","transcript_id","areatr"))
    data2ls[[i]] = isosigi
}
names(data2ls) = names(isosigls.t44)
head(data2ls$layer)

,gene_id,transcript_id,areatr,sig,pvalue,padj,count.M1-1,genecount.M1-1,relative.M1-1,otherrelative.M1-1,⋯,count.M1-2,genecount.M1-2,relative.M1-2,otherrelative.M1-2,dr.M1-2,count.M1-3,genecount.M1-3,relative.M1-3,otherrelative.M1-3,dr.M1-3
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,ACP1,XM_005576671.2,l2,Up,2.200806e-06,4.188992e-05,159,439,0.3621868,0.2617331,⋯,26,54,0.4814815,0.45376915,0.02771233,109,173,0.6300578,0.46530199,0.1647558176
2,ACP1,XM_005576671.2,l3,N,2.200806e-06,4.188992e-05,565,1952,0.2894467,0.2762811,⋯,130,300,0.4333333,0.46580618,-0.03247285,240,530,0.4528302,0.50074751,-0.0479173201
3,ACP1,XM_005576671.2,l4,N,2.200806e-06,4.188992e-05,72,301,0.2392027,0.2863299,⋯,10,22,0.4545455,0.46050315,-0.00595770,28,74,0.3783784,0.51563787,-0.1372594925
4,ACP1,XM_005576671.2,l5,N,2.200806e-06,4.188992e-05,182,781,0.2330346,0.2875635,⋯,50,104,0.4807692,0.45394721,0.02682202,144,276,0.5217391,0.48696572,0.0347734100
5,ACP1,XM_005576671.2,l6,N,2.200806e-06,4.188992e-05,194,758,0.2559367,0.2829831,⋯,50,112,0.4464286,0.46253238,-0.01610380,115,233,0.4935622,0.49260110,0.0009611316
6,AIDA,NM_001283182.1,l3,N,2.210511e-04,2.211496e-03,577,649,0.8890601,0.8432234,⋯,2,32,0.0625000,0.09090909,-0.02840909,11,88,0.1250000,0.08663835,0.0383616514


In [40]:
writexl::write_xlsx(data2ls, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables2_26_01_07.xlsx")

# Table3 Cell type analysis

In [51]:
datas3 = list()

In [62]:
cellst.t42 = fsraw.t42 %>% group_by(layer,region,class,subclass) %>% summarise(celln = length(unique(cell_id)))
cellst.t44 = fsraw.t44 %>% group_by(layer,region,class,subclass) %>% summarise(celln = length(unique(cell_id)))
cellst.t47 = fsraw.t47 %>% group_by(layer,region,class,subclass) %>% summarise(celln = length(unique(cell_label)))
cellst.t42$sample = "Macaque1-2"
cellst.t44$sample = "Macaque1-1"
cellst.t47$sample = "Macaque1-3"
datas3$cellst = rbind(cellst.t42,cellst.t44,cellst.t47)

`summarise()` has grouped output by 'layer', 'region', 'class'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'layer', 'region', 'class'. You can
override using the `.groups` argument.
`summarise()` has grouped output by 'layer', 'region', 'class'. You can
override using the `.groups` argument.


In [53]:
library(readxl)
file_path <- "result/figure_revision/Hubgene_by_subclass_25_11_28.xlsx"
sheet_names <- excel_sheets(file_path)
print(sheet_names)
gofls = list.files("result/figure_revision/GO_analysis/",full.names = T)
gofls = gofls[order(gofls)]

MyGOFilter = function(godfi,ni){
    godfi$gene_ratio = godfi$intersection_size/godfi$term_size
    # godfi = godfi[godfi$intersection_size > 2,]
    # godfi = godfi[grepl(godfi$term_name,pattern = "synap|splic|dendrite") | godfi$source %in% c("HP","KEGG"),]
    godfi = godfi %>% group_by(source) %>% arrange(intersection_size) %>% 
    top_n(wt = intersection_size,n = ni)
    godfi$term_name = factor(godfi$term_name,levels = unique(godfi$term_name))
    print(nrow(godfi))
    godfi = godfi[!godfi$term_name %in% c("cell junction","cytoplasm","cytosol","cytoskeleton"),]
    return(godfi)
}

godf = NULL
for(i in 1:length(gofls)){
    godfi = read.csv(gofls[i])
    godfi = tryCatch(MyGOFilter(godfi,5),error = function(e) 0)
    if(!is.null(nrow(godfi))){
        if(nrow(godfi) > 0){
            godfi$areatr = sheet_names[i]
            godf = rbind(godf,godfi)
        }
    }
}
head(godf)
datas3$godf = godf

 [1] "L2"       "L2_3"     "L2_3_4"   "L3_4"     "L3_4_5"   "L4"      
 [7] "L4_5"     "L4_5_6"   "L5_6"     "L6"       "LAMP5"    "RELN"    
[13] "VIP_RELN" "VIP"      "PVALB"    "SST"      "ASC"      "OPC"     
[19] "OLG"      "MG"      
[1] 13
[1] 5
[1] 11
[1] 2
[1] 8
[1] 4
[1] 3
[1] 3
[1] 9
[1] 3
[1] 2
[1] 13
[1] 9
[1] 3
[1] 9
[1] 7
[1] 16
[1] 10
[1] 11
[1] 9


source,term_name,term_id,highlighted,adjusted_p_value,negative_log10_of_adjusted_p_value,term_size,query_size,intersection_size,effective_domain_size,intersections,gene_ratio,areatr
<chr>,<fct>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<chr>
GO:MF,"DNA exonuclease activity, producing 5'-phosphomonoesters",GO:0016895,false,0.013367903,1.873937,17,173,4,15155,"APTX,EXD2,POLG,TATDN1",0.23529412,L2
GO:MF,DNA exonuclease activity,GO:0004529,false,0.017034582,1.768669,18,173,4,15155,"APTX,EXD2,POLG,TATDN1",0.22222222,L2
GO:MF,exonuclease activity,GO:0004527,true,0.005631442,2.249380,70,173,7,15155,"APTX,CPSF3,EXD2,POLG,RAD1,TATDN1,XRN2",0.10000000,L2
HP,Neonatal hypotonia,HP:0001319,false,0.026389640,1.578567,180,63,11,4759,"AP4B1,BIN1,EGR2,EIF5A,NFIX,NTNG1,PDHX,SLC25A22,SPATA5,SYT1,TPM2",0.06111111,L2
GO:BP,intracellular transport,GO:0046907,false,0.017172569,1.765165,993,180,25,18644,"AGK,AP3S1,AP4B1,ARFIP2,ATF2,BIN1,BLOC1S6,CCDC91,CMTM6,DYNC1I1,EGR2,GOSR1,IFT20,IFT22,M6PR,OS9,PEX5L,PICK1,PRICKLE1,RABGEF1,RAMP3,REPS2,SARNP,SRSF10,TRMT10B",0.02517623,L2
GO:BP,establishment of localization in cell,GO:0051649,true,0.008733240,2.058825,1419,180,32,18644,"AGK,AP3S1,AP4B1,ARFIP2,ATF2,BIN1,BLOC1S6,CACNB4,CCDC91,CEP19,CMTM6,CPLX2,DYNC1I1,EGR2,GOSR1,IFT20,IFT22,M6PR,OS9,PDZD7,PEX5L,PICK1,PRICKLE1,RABGEF1,RACGAP1,RAMP3,REPS2,SARNP,SKA2,SRSF10,SYT1,TRMT10B",0.02255109,L2


In [57]:
isosig.b62 = qread(file = "data/fullscope_qc/Mouse_ST_B62_fullscope_DTU_sc26_01_03.qs")
isosigref.ls = qread("data/fullscope_qc/Mouse_SC_tilgner_fullscope_witnan_26_01_03.qs")
isosigref.cl = isosigref.ls$class
isosigi1 = isosig.b62
isosigi2 = isosigref.cl
renamedf = data.frame("areatr" = c("InhibNeuron","ExciteNeuron","NN"),new = c("GABA","GLU","NN"))
isosigi2 = merge(isosigi2,renamedf,by = "areatr")
isosigi2$areatr = isosigi2$new
# isosigi1 = isosigi1[isosigi1$count > 50,]
# isosigi2 = isosigi2[isosigi2$count > 50,]
hubtrans = unique(c(isosigi1[isosigi1$sig != "N",]$transcript_id,
                   isosigi2[isosigi2$sig != "N",]$transcript_id))
isosigi1 = isosigi1[isosigi1$transcript_id %in% hubtrans & isosigi1$count > 50,]
isosigi2 = isosigi2[isosigi2$transcript_id %in% hubtrans & isosigi2$count > 50,]
isosig.cmp2 = merge(isosigi1[,c("gene_id","transcript_id","areatr","count","dr")],
                   isosigi2[,c("gene_id","transcript_id","areatr","count","dr")],
                   by = c("gene_id","transcript_id","areatr"),suffixes = c(".Fullscope",".Tilgner_sc"))
datas3$mouse_cmp = isosig.cmp2

In [55]:
nrow(datas3$cellanst)

NULL

In [63]:
writexl::write_xlsx(datas3, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables3_26_01_07.xlsx")

# Table4 GLM model

In [66]:
datas4 = list()
modelcoef2 = qread("data/dtu_analysis/region_subclass_isoform_model_tri_onlyregion_25_12_03.qs")

modelcoef2$group = modelcoef2$areatr
modelcoef2[modelcoef2$areatr == "cellm",]$group = "cell"

modelcoef2[modelcoef2$areatr == "regionm",]$group = "region"
head(modelcoef2)

colnames(modelcoef2)[4] = "pvalue"
modelcoef.hub = modelcoef2[modelcoef2$group %in% c("cell","region"),]
length(unique(modelcoef.hub$gene_id))
modelcoef.hub$pvalue = -log10(modelcoef.hub$pvalue)
modelmx1 = dcast(modelcoef.hub,gene_id~group,value.var = "pvalue",fun.aggregate = sum)
modelmx1.hub = modelmx1
modelmx1.hub$sig <- sapply(1:nrow(modelmx1.hub), function(i) {
  significant_cols <- names(modelmx1.hub)[-1][modelmx1.hub[i, -1] > -log10(0.05)]
  paste(significant_cols, collapse = "-")
})
datas4$region_model = modelmx1.hub

,Estimate,Std. Error,z value,pvalue,gene_id,areatr,group
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
(Intercept),-59.727097,2366015078,-2.524375e-08,1,ABI2,(Intercept),(Intercept)
cellm,53.450647,1992326184,2.682826e-08,1,ABI2,cellm,cell
regionm,53.214197,1100514613,4.835392e-08,1,ABI2,regionm,region
(Intercept)1,-5.806286,771268832,-7.528226e-09,1,ADD1,(Intercept),(Intercept)
cellm1,30.355730,567226468,5.351607e-08,1,ADD1,cellm,cell
regionm1,21.291255,548798317,3.879614e-08,1,ADD1,regionm,region


[1] 354

In [67]:
writexl::write_xlsx(datas4, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables4_26_01_07.xlsx")

# Table5

In [68]:
library(dplyr)
library(tidyr)
library(purrr)
# major isoform shift stat
isosigi = isosigls.t44$subcortical
hubgene = unique(isosigi[isosigi$sig != "N",]$gene_id)
isosigi = isosigi[isosigi$gene_id %in% hubgene,]
head(isosigi)
#construct phase dataset
isosigi$ismajor <- FALSE
isosigi <- isosigi %>%
  group_by(areatr, gene_id) %>%
  mutate(ismajor = ifelse(relative == max(relative), TRUE, FALSE)) %>%
  ungroup()
isosigi = isosigi %>% group_by(gene_id) %>% filter(meanrelative == max(meanrelative))
#isosigi[isosigi$relative > 0.5,]$ismajor = TRUE
isosigi = isosigi[isosigi$ismajor == TRUE | (isosigi$ismajor == FALSE & abs(isosigi$relative - isosigi$otherrelative) > 0.1),]
isosigi = isosigi %>% group_by(transcript_id) %>% filter(length(unique(ismajor)) > 1)
length(unique(isosigi$gene_id))
length(unique(isosigi$transcript_id))
library(tidyr)
df_wide <- pivot_wider(isosigi, 
                       id_cols = transcript_id, 
                       names_from = areatr, 
                       values_from = ismajor)


Attaching package: 'tidyr'


The following object is masked from 'package:reshape2':

    smiths


The following object is masked from 'package:S4Vectors':

    expand



Attaching package: 'purrr'


The following object is masked from 'package:XVector':

    compact


The following object is masked from 'package:GenomicRanges':

    reduce


The following object is masked from 'package:IRanges':

    reduce


The following object is masked from 'package:data.table':

    transpose




,gene_id,transcript_id,areatr,count,genecount,relative,meanrelative,otherrelative,dr,sig,pvalue,padj
,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
9,AACS,NM_001284010.1,Cerebellum,146,158,0.9240506,0.855632,0.8419483,0.08210238,N,0.001852607,0.005294328
10,AACS,NM_001284010.1,Cortex,2879,3272,0.8798900,0.855632,0.8507804,0.02910959,N,0.001852607,0.005294328
11,AACS,NM_001284010.1,Hippocampus,92,117,0.7863248,0.855632,0.8694934,-0.08316864,N,0.001852607,0.005294328
12,AACS,NM_001284010.1,Midbrain,37,48,0.7708333,0.855632,0.8725917,-0.10175838,Down,0.001852607,0.005294328
13,AACS,NM_001284010.1,Striatum,29,31,0.9354839,0.855632,0.8396616,0.09582226,N,0.001852607,0.005294328
14,AACS,NM_001284010.1,Ventricle,36,43,0.8372093,0.855632,0.8593165,-0.02210722,N,0.001852607,0.005294328


[1] 375

[1] 375

In [72]:
datas5 = list()
datas5$shift_isoform = isosigi
datas5$shift_isoform_st = df_wide

In [75]:
transt = qread(file = "data/cross_region/transt_subcortical_cell_level_25_10_28.qs")
transt = transt[transt$transcript_id %in% isosigi$transcript_id,]
nrow(transt)

[1] 4515

In [76]:
datas5$celldtu = transt

In [77]:
writexl::write_xlsx(datas5, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables5_26_01_07.xlsx")

# Table 6

In [82]:
refgdf = read.csv("ref/disease_gene/Total_gene_list_summary_25_03_19.csv")
head(refgdf)

,gene_id,source,group
,<chr>,<chr>,<chr>
1,MED13,ASD.Talkowski,ASD
2,SYNGAP1,ASD.Talkowski,ASD
3,TRIP12,ASD.Talkowski,ASD
4,GRIN2B,ASD.Talkowski,ASD
5,ASH1L,ASD.Talkowski,ASD
6,MED13L,ASD.Talkowski,ASD


In [84]:
isosigls = isosigls.t44
# gtf.hp = rtracklayer::import("ref/cross_species/gencode.v19.annotation.gtf")
# gtf = rtracklayer::import("../ref/bambu_extended_annotations_novelname.gtf")
cmgeneid = intersect(gtf.hp$gene_name,gtf$gene_id)
cmgeneid = cmgeneid[cmgeneid %in% fsraw.t44$gene_id]
cmgenen = length(cmgeneid)
cmgenen

[1] 11982

In [86]:
hubsource = c('ASD.DTU','BD.DTU','SCZ.DTU','ASD.GWAS','BD.GWAS','SCZ.GWAS','NDD.GWAS','DDD.GWAS',
              'BIP.GWAS','EPI.GWAS','ASD.TWAS','BD.TWAS',"SCZ.TWAS")
FishEnrich = function(geneset,funcset,total_genes){
    drugovl = intersect(unique(funcset),geneset)
    geneset_size <- length(geneset)    # 感兴趣的基因集大小
    functional_category_size <- length(unique(funcset))  # 特定功能类别中的基因总数
    both <- length(drugovl)  # 两个集合都包含的基因数量
    #total_genes <- 20000  # 整个基因组中的基因总数

    contingency_table <- matrix(c(both, geneset_size - both, 
                              functional_category_size - both, 
                              total_genes - (geneset_size + functional_category_size - both)), 
                            nrow = 2)

    # 进行Fisher精确检验
    fisher_test_result <- fisher.test(contingency_table)
    # 输出p值
    p_value <- fisher_test_result$p.value

    # 计算富集分数（Enrichment Ratio），这通常是感兴趣的基因集中具有特定功能的基因比例与整个基因组中该功能基因比例的比值
    enrichment_ratio <- (both / geneset_size) / (functional_category_size / total_genes)
    res = data.frame("p.value" = p_value,"enrichscore" = enrichment_ratio,"overlap_size" = both,
                     "term_size" = functional_category_size,"query_size" = geneset_size,
                    "overlap_ratio" = both/functional_category_size)
    return(res)
}

DiseaseEnrich = function(isosigt,refgdf){
    isosigt = isosigt[substr(isosigt$gene_id,1,9) != "BambuGene",]
    areatrs = unique(isosigt$areatr)
    goana = NULL
    refgdfi = refgdf
    sourceid = unique(refgdfi$source)
    for(i in 1:length(areatrs)){
        isosigii = isosigt[isosigt$areatr == areatrs[i],]
        hubgene = unique(isosigii$gene_id)
        for(j in 1:length(sourceid)){
            refgene = unique(refgdfi[refgdfi$source == sourceid[j],]$gene_id)
            goanai = FishEnrich(hubgene,refgene,cmgenen)
            goanai$group = sourceid[j]
            goanai$areatr = areatrs[i]
            goana = rbind(goana,goanai)
        }
    }
    goana$fdr = p.adjust(goana$p.value)
    goanaf = goana[goana$group %in% hubsource,]
    #goanaf
    #goanaf$group = MySplit(goanaf$group,"\\.",1)
    goanaf$group = factor(goanaf$group,levels = hubsource)
    goanaf$stars <- cut(goanaf$p.value, breaks=c(-Inf, 0.001, 0.01, 0.05, Inf), label=c("***", "**", "*", ""))  # Create column of significance labels
    return(goanaf)
}
#subclass
isosigt = isosigls$subclass
isosigt = isosigt[isosigt$sig != "N",]
isosigt = isosigt %>% group_by(transcript_id,sig) %>% filter(length(unique(areatr)) == 1)
isosigt = isosigt[isosigt$gene_id %in% cmgeneid,]
goanaf.subuni = DiseaseEnrich(isosigt,refgdf)
goanaf.subuni$sample = "subclass"
#class
isosigt = isosigls$class
isosigt = isosigt[isosigt$sig != "N",]
isosigt = isosigt %>% group_by(transcript_id,sig) %>% filter(length(unique(areatr)) == 1)
goanaf.class = DiseaseEnrich(isosigt,refgdf)
goanaf.class$sample = "class"

#layer
isosigt = isosigls$layer
isosigt = isosigt[isosigt$sig != "N",]
isosigt = isosigt %>% group_by(transcript_id,sig) %>% filter(length(unique(areatr)) == 1)
goanaf.layer = DiseaseEnrich(isosigt,refgdf)
goanaf.layer$sample = "layer"

#lobe
isosigt = isosigls$lobe
isosigt = isosigt[isosigt$sig != "N",]
isosigt = isosigt %>% group_by(transcript_id,sig) %>% filter(length(unique(areatr)) == 1)
isosigt = isosigt[isosigt$gene_id %in% cmgeneid,]
goanaf.lobe = DiseaseEnrich(isosigt,refgdf)
goanaf.lobe$sample = "lobe"

#subcortical
isosigt = isosigls$subcortical
isosigt = isosigt[isosigt$sig != "N",]
isosigt = isosigt %>% group_by(transcript_id,sig) %>% filter(length(unique(areatr)) == 1)
isosigt = isosigt[isosigt$gene_id %in% cmgeneid,]
goanaf.subc = DiseaseEnrich(isosigt,refgdf)
goanaf.subc$sample = "subcortical"

goanaft = rbind(goanaf.subuni,goanaf.class,goanaf.layer,goanaf.lobe,goanaf.subc)


In [87]:
gtf.fil = rtracklayer::import("ref/cross_species/bambu_extended_annotations_novelname_fil_04_30.gtf")
hgvmac.tmap = read.table("ref/cross_species/bambu_macaca_vs_hg19lifted_25_03_16.gtf.bambu_extended_annotations_novelname_fil_04_30.gtf.tmap",
                        header = T)
hgvmac.tmapf = hgvmac.tmap[hgvmac.tmap$class_code == "=",]
hgvmac.tmapf$ref_id = substr(hgvmac.tmapf$ref_id,1,nchar(hgvmac.tmapf$ref_id)-2)
trnumt = length(unique(gtf.fil$transcript_id))
trnumhit = length(unique(hgvmac.tmapf$qry_id))
trnumhit
trnumhit/trnumt

[1] 20837

[1] 0.2215901

In [89]:
data6s = list()
data6s$ref = refgdf
data6s$enrich = goanaft
data6s$cross_species = hgvmac.tmapf

In [92]:
discor = read_xlsx("result/figure6_snp_analysis//Disease_analysis_of_top_DTUs_by_disgener_25_12_24.xlsx")
data6s$Diseas_score = discor

In [93]:
writexl::write_xlsx(data6s, "/cluster/home/liuhengxin/P1_brain_nanopore/downstream_analysis/result/figure_revision/data_summary/tables6_26_01_07.xlsx")